# Getting Steam store app list
## Based on this article: https://nik-davis.github.io/posts/2019/steam-data-collection/

In [6]:
import csv
import datetime as dt
import os
import statistics
import time

import numpy as np
import pandas as pd
import requests
from requests.exceptions import SSLError


STEAM_API_KEY = 'D518AC94ECCEE3B45FC0DCA2334329B9'


# generic HTTP GET request function
def get_request(url, parameters=None):
	try:
		response = requests.get(url=url, params=parameters)
	except SSLError as s:
		print('SSL Error:', s)
		
		for i in range(5, 0, -1):
			print('\rWaiting... ({})'.format(i), end='')
			time.sleep(1)
		print('\rRetrying.' + ' ' * 10)
		
		# recusively try again
		return get_request(url, parameters)
	
	if response:
		return response.json()
	else:
		# response is none usually means too many requests. Wait and try again
		print('No response, waiting 10 seconds...')
		time.sleep(10)
		print('Retrying.')
		return get_request(url, parameters)



def generate_app_list(printExample = False):
	url = "https://steamspy.com/api.php"
	parameters = {"request": "all"}
	
	# request 'all' from steam spy and parse into dataframe
	json_data = get_request(url, parameters=parameters)
	steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')
	
	# generate sorted app_list from steamspy data
	app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)
	
	# export disabled to keep consistency across download sessions
	app_list.to_csv('./data/download/app_list.csv', index=False)
	
	# instead read from stored csv
	app_list = pd.read_csv('./data/download/app_list.csv')
	
	if printExample:
		print(app_list.head())
	
	return app_list

# Generate Steam app list
app_list = generate_app_list()

   appid                       name
0     10             Counter-Strike
1     20      Team Fortress Classic
2     30              Day of Defeat
3     40         Deathmatch Classic
4     50  Half-Life: Opposing Force


# Get data from each app generated in the `app_list`

In [7]:
def get_app_data(start, stop, parser, pause, app_list):
	"""Return list of app data generated from parser.

	parser : function to handle request
	"""
	app_data = []
	
	# iterate through each row of app_list, confined by start and stop
	for index, row in app_list[start:stop].iterrows():
		print('Current index: {}'.format(index), end='\r')
		
		appid = row['appid']
		name = row['name']
		
		# retrive app data for a row, handled by supplied parser, and append to list
		data = parser(appid, name)
		app_data.append(data)
		
		time.sleep(pause)  # prevent overloading api with requests
	
	return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename, columns, begin=0, end=-1, batchsize=100, pause=1):
	"""Process app data in batches, writing directly to file.

	parser : custom function to format request
	app_list : dataframe of appid and name
	download_path : path to store data
	data_filename : filename to save app data
	index_filename : filename to store highest index written
	columns : column names for file

	Keyword arguments:

	begin : starting index (get from index_filename, default 0)
	end : index to finish (defaults to end of app_list)
	batchsize : number of apps to write in each batch (default 100)
	pause : time to wait after each api request (defualt 1)

	returns: none
	"""
	print('Starting at index {}:\n'.format(begin))
	
	# by default, process all apps in app_list
	if end == -1:
		end = len(app_list) + 1
	
	# generate array of batch begin and end points
	batches = np.arange(begin, end, batchsize)
	batches = np.append(batches, end)
	
	apps_written = 0
	batch_times = []
	
	for i in range(len(batches) - 1):
		start_time = time.time()
		
		start = batches[i]
		stop = batches[i + 1]
		
		app_data = get_app_data(start, stop, parser, pause, app_list)
		
		rel_path = os.path.join(download_path, data_filename)
		
		# writing app data to file
		with open(rel_path, 'a', newline='', encoding='utf-8') as f:
			writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
			
			for j in range(3, 0, -1):
				print("\rAbout to write data, don't stop script! ({})".format(j), end='')
				time.sleep(0.5)
			
			writer.writerows(app_data)
			print('\rExported lines {}-{} to {}.'.format(start, stop - 1, data_filename), end=' ')
		
		apps_written += len(app_data)
		
		idx_path = os.path.join(download_path, index_filename)
		
		# writing last index to file
		with open(idx_path, 'w') as f:
			index = stop
			print(index, file=f)
		
		# logging time taken
		end_time = time.time()
		time_taken = end_time - start_time
		
		batch_times.append(time_taken)
		mean_time = statistics.mean(batch_times)
		
		est_remaining = (len(batches) - i - 2) * mean_time
		
		remaining_td = dt.timedelta(seconds=round(est_remaining))
		time_td = dt.timedelta(seconds=round(time_taken))
		mean_td = dt.timedelta(seconds=round(mean_time))
		
		print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
	
	print('\nProcessing batches complete. {} apps written'.format(apps_written))


def reset_index(download_path, index_filename):
	"""Reset index in file to 0."""
	rel_path = os.path.join(download_path, index_filename)
	
	with open(rel_path, 'w') as f:
		print(0, file=f)


def get_index(download_path, index_filename):
	"""Retrieve index from file, returning 0 if file not found."""
	try:
		rel_path = os.path.join(download_path, index_filename)
		
		with open(rel_path, 'r') as f:
			index = int(f.readline())
	
	except FileNotFoundError:
		index = 0
	
	return index


def prepare_data_file(download_path, filename, index, columns):
	"""Create file and write headers if index is 0."""
	if index == 0:
		rel_path = os.path.join(download_path, filename)
		
		with open(rel_path, 'w', newline='') as f:
			writer = csv.DictWriter(f, fieldnames=columns)
			writer.writeheader()


def parse_steam_request(appid, name):
	"""Unique parser to handle data from Steam Store API.

	Returns : json formatted data (dict-like)
	"""
	url = "http://store.steampowered.com/api/appdetails/"
	parameters = {"appids": appid}
	
	json_data = get_request(url, parameters=parameters)
	json_app_data = json_data[str(appid)]
	
	if json_app_data['success']:
		data = json_app_data['data']
	else:
		data = {'name': name, 'steam_appid': appid}
	
	return data


def generate_app_list():
	url = "https://steamspy.com/api.php"
	parameters = {"request": "all"}
	
	# request 'all' from steam spy and parse into dataframe
	json_data = get_request(url, parameters=parameters)
	steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')
	
	# generate sorted app_list from steamspy data
	app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)
	
	# export disabled to keep consistency across download sessions
	app_list.to_csv('./data/download/app_list.csv', index=False)
	
	# instead read from stored csv
	app_list = pd.read_csv('./data/download/app_list.csv')
	
	# display first few rows
	print(app_list.head())
	
	return app_list

# Set file parameters
download_path = './data/download'
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_index.txt'

steam_columns = [
	'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
	'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
	'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
	'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
	'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
	'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
	'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
	'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
	parser=parse_steam_request,
	app_list=app_list,
	download_path=download_path,
	data_filename=steam_app_data,
	index_filename=steam_index,
	columns=steam_columns,
	# remove next 3 lines to get data for all apps in app_list
	begin=index,
	end=10,
	batchsize=5
)

print(pd.read_csv('./data/download/steam_app_data.csv').head())

Starting at index 0:

Exported lines 0-4 to steam_app_data.csv. Batch 0 time: 0:00:08 (avg: 0:00:08, remaining: 0:00:08)
Exported lines 5-9 to steam_app_data.csv. Batch 1 time: 0:00:08 (avg: 0:00:08, remaining: 0:00:00)

Processing batches complete. 10 apps written
   type                       name  steam_appid  required_age  is_free  \
0  game             Counter-Strike           10             0    False   
1  game      Team Fortress Classic           20             0    False   
2  game              Day of Defeat           30             0    False   
3  game         Deathmatch Classic           40             0    False   
4  game  Half-Life: Opposing Force           50             0    False   

   controller_support  dlc                               detailed_description  \
0                 NaN  NaN  Zahrajte si nejoblíbenější akční hru na světě....   
1                 NaN  NaN  One of the most popular online action games of...   
2                 NaN  NaN  Enlist in an inten